# <span style="color:#cc5500">Introduction</span>

Credit and thanks to Paul Randal for the content of this chapter.

Corruption does happen, many times per day, all around the world

Check out the SQL Server [SQLTeam.com Forums](https://forums.sqlteam.com/) data corruption forums.  Many people don’t realize they have corruption until too late. Either they don’t know how to check for corruption or they miss the warning signs that corruption has occurred.  The faster you realize you have corruption, the more likely you will be able to recover with minimal downtime and data loss

<span style="color: rgb(204, 85, 0);">Why is this a critical subject to learn?</span>

Many people don't realize they have corruption until it is too late.  Either they don't know how to check for corruption or they miss the warning signs that corruption has occured.  The faster you realize you have corruption, the more likely you will be able to recover with minimal downtime and dataloss.  

Many people don’t know what to do when they do have corruption, leading to:

- Additional data loss than necessary
- Additional downtime than necessary
- Monetary penalties and even job losses

### <span style="color:#cc5500">Things that People many times do first</span>

- Panic
    - This causes adrenaline to flow, can cause you to make a rash decision, or succomb to pressure for others
- Restart SQL Server
    - Just wastes time and delays getting back online
    - Especially if the physical server is rebooted too
- Immediately jump to the “last resort” and then cause further data loss without working through options
    - E.g. running repair instead of using backups
    - E.g. removing the transaction log
- Forcibly detach/drop a damaged or SUSPECT database
    - it will fail to attach again so now you have made the situation is even worse

### <span style="color:#cc5500">Practice Makes Perfect</span>

If you’re recovering from corruption, or helping a client recover from some corruption:

- You must know what you’re doing
- You must have practiced
- You must NOT make things worse by making mistakes

Even if you’ve never run consistency checks before, a little bit of knowledge means you’ll have a good idea what to do and when to ask for help from someone more experienced.

### <span style="color:#cc5500">Root Cause Analysis</span>

No matter what method you use to recover from corruption, you should always determine why it happened to avoid future problems.  It may be obvious what happened.  For example a known SAN problem or power failure.  It also be possible that you have have no idea what happened, so what to do?

- Bing/Google for the corruption message you saw
- Run I/O subsystem and server memory diagnostics
- Examine the SQL Server error log and Windows event logs for clues
- Check that firmware is up-to-date
- Investigate NTFS filter drivers
- Possibly contact Microsoft Customer Support for assistance

### <span style="color:#cc5500">How Does Corruption Occur?</span>

If a data file page is “good” when written out of SQL Server’s memory, but “bad” when read back into memory, that’s corruption.  Where these is code, there is the potential for bugs.  There are a lot of moving 'parts' underneath SQL Server.  Transaction Log files can become currupt too, but this scenario will be covered in a later notebook.  Here are the most common causes of Corruption?

- It’s almost always the I/O subsystem
- The I/O subsystem means anything “underneath” SQL Server
    - Windows operating system
    - File system filter drivers
    - Antivirus, defraggers, encryption
    - Network cards, switches, cables
    - SAN controllers
    - RAID controllers
    - Disks
- Consider the Mean Time Between Failure (MTBF) of disks
    - Various sources online give figures around 1.5 million hours as MTBFas long as the operating environment is ideal
    - Dust, high/low temperature, vibration, power cycling are not ideal
    - It’s a mean, which means there’s a bell-curve of failure times.  Likely you will find yourself on the low end of the bell curve

Jim Gray likened a disk head in a 15,000 rpm disk to a Boeing 747 flying at 500mph about ¼ inch above the ground.  What are the tolerances involved?  What happens in a crash?  A disk head is essentially floating on a couple of microns of air above the disk and the slightest vibration or dust can cause corruption.

What if it’s not the I/O subsystem?  Here are a few more examples:

- Memory corruption
- Bad memory chips
- Memory Scribblers
- SQL Server bugs
- Human error
- Manually trying to edit a database
- Manually deleting a transaction log

### <span style="color:#cc5500">What does not cause corruption?</span>

There are many myths and misconceptions about what can cause database corruption.  Corruptions are not caused by:

- Anything an application can do (unless it is malicious code from a bad actor)
- Anything you can do in SQL Server with supported, documented commands
- Interrupting a database shrink, index rebuild, or long-running batch.   SQL Server is designed to 'crash' and self-recover from it by means of the transaction log
- Shutting down SQL Server

### <span style="color:#cc5500">What about propogation of Corruption to remote servers?</span>

- SQL Server redundancy technologies do not propagate data file corruptions caused by the I/O subsystem
    - Database mirroring, log shipping, replication, Always On Availability Groups
- These technologies send log records (or actions based on log records), not data file pages from disk
- It is possible to have a “second-order” corruption propagated
    - Corrupt page is not detected on the local server and used to calculate a result which is stored locally, and sent to the remote server
- Backing up a database should detect corruption, preventing propagation to a remote server during restore
- SAN replication should also not propagate corruption
    - Replicating changed disk blocks as they’re written, not read from disk

### <span style="color:#cc5500">What is Disappearing Corruption?</span>

- Typical description of the phenomenon
    - Consistency-checking job fails during the night
    - DBA runs a consistency check again in the morning, but no corruption!
    - DBA questions where the initial report of corruption was correct
- What’s really happening:
    - Corruption is reported on some pages allocated to an index for instance
    - After the consistency-checking job runs, an index maintenance job runs
    - The index containing the corrupt pages is rebuilt, which builds a new index with new data pages, and then de-allocates the old index and data pages
    - Another consistency check runs but doesn’t find any corruption
    - The second consistency check is reading different pages from the first one, so doesn’t see the corrupt pages
- Consistency checks validate those data file pages that are allocated to tables and indexes in the database